In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
import pandas as pd
import numpy as np
Indonesia=pd.read_csv('https://github.com/PeishanLi/G5055_Practicum_Project2/raw/main/Data/Indonesia.csv')
Guatemala=pd.read_csv('https://github.com/PeishanLi/G5055_Practicum_Project2/raw/main/Data/Guatemala.csv')
print(Indonesia.size)
print(Guatemala.size)

270912
306880


In [45]:
#Eliminate disaggregation rows from dataset
Indonesia=Indonesia[Indonesia['SeriesDescription'].str.contains('by')==False]
Guatemala=Guatemala[Guatemala['SeriesDescription'].str.contains('by')==False]
print(Indonesia.size)
print(Guatemala.size)

115904
122880


In [46]:
def preprocess_for_correlation(df): 
    df_copy=df.copy()

    # If a column with unique identifying info has nothing, we then will have blanks for those fields - otherwise concatenating the text will be NanNan etc. 
    df_copy=df_copy.fillna('')

    # Concatenated Column with a number of identifiers 
    df_copy['UniqueID']=df_copy[['SeriesCode','[Sex]',\
                                   '[Deviation Level]', '[Mountain Elevation]',\
                                   '[Parliamentary committees]', '[Mode of transportation]',\
                                   '[Type of speed]', '[Policy instruments]', '[Type of skill]',\
                                   '[Education level]', '[Location]', '[Food Waste Sector]',\
                                   '[Freq]', '[Type of product]', '[Observation Status]',\
                                   '[Type of occupation]','[Name of non-communicable disease]', '[Level/Status]',\
                                   '[Age]', '[Disability status]','[Frequency of Chlorophyll-a concentration]',\
                                   '[Activity]', '[Level of requirement]', '[Quantile]',\
                                   '[IHR Capacity]','[Name of international institution]'\
                                  ]].apply(lambda x: ' '.join(x), axis=1)
    duplicates=df_copy[df_copy.duplicated(subset=['UniqueID','Goal', 'Target', 'Indicator',\
                                      'SeriesCode', 'SeriesDescription','Source','TimePeriod'])]
    duplicates # if you print duplicates (outside of this function) you will find a small number of duplicate columns; 
    # however when I looked at them, the values were largely consistent - the only differing column I could find was Value. 
    # Let me know if you think this should be done differently. 
    
    # For the time being I will remove columns that are duplicates across all columns (less that of 'Values')
    df_copy=df_copy[['UniqueID','Source','Goal', 'Target', 'Indicator', 'SeriesCode', 'SeriesDescription','[Units]','[Nature]',
       'GeoAreaCode', 'GeoAreaName', 'Time_Detail', 'Value','[Reporting Type]','TimePeriod']].drop_duplicates(subset=['UniqueID','Source','Goal', 'Target', 'Indicator', 'SeriesCode', 'SeriesDescription','[Units]','[Nature]',
       'GeoAreaCode', 'GeoAreaName', 'Time_Detail','[Reporting Type]','TimePeriod'])
    df_copy=df_copy[['UniqueID','Goal', 'Target', 'Indicator', 'SeriesCode','TimePeriod','Value']].drop_duplicates(subset=['UniqueID', 'Goal', 'Target', 'Indicator', 'SeriesCode', 'TimePeriod'])   
    df_copy=df_copy.dropna()
    df_copy['Value']=pd.to_numeric(df_copy['Value'])
    return df_copy

In [49]:
processedIndo=preprocess_for_correlation(Indonesia)
print(processedIndo)
processedIndo.to_csv('/content/drive/MyDrive/Colab Notebooks/Practicum SDG Networks/Data/processedIndo.csv')

                                   UniqueID  Goal  ... TimePeriod       Value
0      SI_POV_DAY1                            1.0  ...     2012.0     9.50000
1      SI_POV_DAY1                            1.0  ...     2013.0     7.30000
2      SI_POV_DAY1                            1.0  ...     2014.0     6.20000
3      SI_POV_DAY1                            1.0  ...     2015.0     5.80000
4      SI_POV_DAY1                            1.0  ...     2016.0     5.20000
...                                     ...   ...  ...        ...         ...
4227  DC_FTA_TOTAL                           17.0  ...     2015.0  3409.18378
4228  DC_FTA_TOTAL                           17.0  ...     2016.0  1053.97416
4229  DC_FTA_TOTAL                           17.0  ...     2017.0  1168.49150
4230  DC_FTA_TOTAL                           17.0  ...     2018.0  2042.80551
4231  DC_FTA_TOTAL                           17.0  ...     2019.0   886.47354

[1809 rows x 7 columns]


In [50]:
processedIndo['UniqueID'].head(30)

0             SI_POV_DAY1                         
1             SI_POV_DAY1                         
2             SI_POV_DAY1                         
3             SI_POV_DAY1                         
4             SI_POV_DAY1                         
5             SI_POV_DAY1                         
6             SI_POV_DAY1                         
7             SI_POV_DAY1                         
71     SI_POV_NAHC          ALLAREA               
72     SI_POV_NAHC          ALLAREA               
73     SI_POV_NAHC          ALLAREA               
74     SI_COV_MATNL FEMALE                        
162          VC_DSR_GDPLS                         
163          VC_DSR_GDPLS                         
164          VC_DSR_GDPLS                         
165          VC_DSR_GDPLS                         
166          VC_DSR_GDPLS                         
167          VC_DSR_GDPLS                         
168          VC_DSR_GDPLS                         
169          VC_DSR_GDPLS      

In [ ]:
'''def measures_each_indicator(df):
  Allindicators=df['Indicator'].drop_duplicates()
  for indicator in Allindicators:
    AllID=list(df['UniqueID'].drop_duplicates())
    if len(AllID)>1:
      group=df[df['Indicator']==indicator]
      group=group.drop(columns=['TimePeriod','Value'],axis=1)
      path='/content/drive/MyDrive/Colab Notebooks/Practicum SDG Networks/Data/Indonesia Correlation among measurements/Attribute '+indicator+'.csv'
      group.to_csv(path)
      print('Attribute '+indicator+' completed')'''

In [51]:
#measures_each_indicator(processedIndo)

In [52]:
def calculate_correlation(df):
  Allindicators=df['Indicator'].drop_duplicates()
  for indicator in Allindicators:
    AllID=list(df['UniqueID'].drop_duplicates())
    if len(AllID)>1:
      group=df[df['Indicator']==indicator]
      group=group.drop(columns=['Goal','Target','Indicator','SeriesCode'],axis=1)
      group_pivot=group.pivot(index=['TimePeriod'], columns=['UniqueID'], values='Value').reset_index()
      group_pivot=group_pivot.apply(lambda x:x.astype(float))
      correlation=group_pivot.corr()
      correlation=pd.DataFrame(correlation)
      correlation=correlation.drop('TimePeriod', axis=1)
      correlation=correlation.drop('TimePeriod', axis=0)
      path='/content/drive/MyDrive/Colab Notebooks/Practicum SDG Networks/Data/Indonesia Correlation among measurements/Indicator '+indicator+'.csv'
      correlation.to_csv(path)
      print('Indicator '+indicator+' completed')

In [53]:
calculate_correlation(processedIndo)

Indicator 1.1.1 completed
Indicator 1.2.1 completed
Indicator 1.3.1 completed
Indicator 1.5.2 completed
Indicator 1.5.1 completed
Indicator 1.5.3 completed
Indicator 1.5.4 completed
Indicator 1.a.2 completed
Indicator 2.1.1 completed
Indicator 2.1.2 completed
Indicator 2.2.1 completed
Indicator 2.2.2 completed
Indicator 2.2.3 completed
Indicator 2.3.1 completed
Indicator 2.3.2 completed
Indicator 2.5.1 completed
Indicator 2.5.2 completed
Indicator 2.a.1 completed
Indicator 2.b.1 completed
Indicator 2.c.1 completed
Indicator 3.1.1 completed
Indicator 3.2.1 completed
Indicator 3.2.2 completed
Indicator 3.3.3 completed
Indicator 3.3.5 completed
Indicator 3.3.2 completed
Indicator 3.3.4 completed
Indicator 3.4.1 completed
Indicator 3.5.2 completed
Indicator 3.5.1 completed
Indicator 3.7.1 completed
Indicator 3.7.2 completed
Indicator 3.8.2 completed
Indicator 3.8.1 completed
Indicator 3.9.1 completed
Indicator 3.9.2 completed
Indicator 3.b.1 completed
Indicator 3.d.2 completed
Indicator 4.

In [56]:
processedGuate=preprocess_for_correlation(Guatemala)
print(processedGuate)
#processedGuate.to_csv('/content/drive/MyDrive/Colab Notebooks/Practicum SDG Networks/Data/processedGuate.csv')

                                               UniqueID  ...      Value
0                  SI_POV_DAY1                           ...    8.80000
64            SI_POV_NAHC          URBAN                 ...   42.20000
65            SI_POV_NAHC          RURAL                 ...   76.10000
66          SI_POV_NAHC          ALLAREA                 ...   59.30000
67    SD_MDP_MUHC BOTHSEX         ALLAREA        ALL...  ...   61.60000
...                                                 ...  ...        ...
4789              DC_FTA_TOTAL                           ...   48.66068
4790              DC_FTA_TOTAL                           ...   77.71732
4791              DC_FTA_TOTAL                           ...   94.32348
4792              DC_FTA_TOTAL                           ...  289.73249
4793              DC_FTA_TOTAL                           ...   61.08202

[1918 rows x 7 columns]


In [ ]:
#measures_each_indicator(processedGuate)

In [57]:
def calculate_correlation(df):
  Allindicators=df['Indicator'].drop_duplicates()
  for indicator in Allindicators:
    AllID=list(df['UniqueID'].drop_duplicates())
    if len(AllID)>1:
      group=df[df['Indicator']==indicator]
      group=group.drop(columns=['Goal','Target','Indicator','SeriesCode'],axis=1)
      group_pivot=group.pivot(index=['TimePeriod'],columns=['UniqueID'],values='Value').reset_index()
      group_pivot=group_pivot.apply(lambda x:x.astype(float))
      correlation=group_pivot.corr()
      correlation=correlation.drop('TimePeriod', axis=1)
      correlation=correlation.drop('TimePeriod', axis=0)
      path='/content/drive/MyDrive/Colab Notebooks/Practicum SDG Networks/Data/Guatemala Correlation among measurements/Indicator '+indicator+'.csv'
      correlation.to_csv(path)
      print('Indicator '+indicator+' completed')

In [58]:
calculate_correlation(processedGuate)

Indicator 1.1.1 completed
Indicator 1.2.1 completed
Indicator 1.2.2 completed
Indicator 1.3.1 completed
Indicator 1.5.2 completed
Indicator 1.5.1 completed
Indicator 1.5.3 completed
Indicator 1.5.4 completed
Indicator 1.a.2 completed
Indicator 2.1.1 completed
Indicator 2.1.2 completed
Indicator 2.2.1 completed
Indicator 2.2.2 completed
Indicator 2.2.3 completed
Indicator 2.3.1 completed
Indicator 2.3.2 completed
Indicator 2.5.1 completed
Indicator 2.5.2 completed
Indicator 2.a.1 completed
Indicator 2.c.1 completed
Indicator 3.1.1 completed
Indicator 3.2.1 completed
Indicator 3.2.2 completed
Indicator 3.3.3 completed
Indicator 3.3.5 completed
Indicator 3.3.2 completed
Indicator 3.3.4 completed
Indicator 3.4.1 completed
Indicator 3.5.2 completed
Indicator 3.5.1 completed
Indicator 3.7.1 completed
Indicator 3.7.2 completed
Indicator 3.8.2 completed
Indicator 3.8.1 completed
Indicator 3.9.1 completed
Indicator 3.9.2 completed
Indicator 3.b.1 completed
Indicator 4.1.1 completed
Indicator 4.